## This notebook will be used for the coursera Data Science Capstone Project

In [4]:
#import libraries:
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!pip install folium
import folium # map rendering library

### Part 1: Web scrapping and cleaning:

In [61]:
#get data directly from wiki page:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [62]:
#check df:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [63]:
df.shape

(180, 3)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


#### Notes after assessment:

- There are no missing values and all dtypes are correct.

- However, we only want rows where there is an assigned Borough, therefore, we need to identify and assess how many rows have Borough == 'Not assigned'

In [64]:
#boroughs == 'Not assigned'?
len(df[df.Borough == 'Not assigned'])

77

In [23]:
#calculate n rows after the above rows are dropped:
180 - 77

103

**There are 77 rows where Borough != 'Not assigned', we need to ignore rows with a borough that is Not assigned, thus, these rows need to be dropped**

In [65]:
#drop rows where boroughs == 'Not assigned':

df = df[df.Borough != 'Not assigned']

In [66]:
#test change:
df.shape

(103, 3)

In [21]:
#test:
df[df.Borough == 'Not assigned']

,Postal Code,Borough,Neighbourhood


**The change was successful - all rows with a borough that is Not assigned have been removed from the df.**

--------------------------------

**Check whether or not there are rows where Neighbourhood == 'Not assigned' in the df**

In [22]:
#Neighbourhood == 'Not assigned'?
df[df.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [31]:
pd.set_option('display.max_rows', None)
#df.Neighbourhood.value_counts()
df[df.Neighbourhood.str.contains("Not")]

,Postal Code,Borough,Neighbourhood


**There are no rows where Neighbourhood == 'Not assigned', therefore, no further cleaning needs to be performed on the df.**

In [33]:
print('After cleaning the dataset, there are {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

After cleaning the dataset, there are 103 rows and 3 columns.


----------------------

### Part 2: Get the latitude and the longitude coordinates of each neighborhood. 

In [36]:
print(df['Postal Code'].nunique())
df['Postal Code'].unique()

103


array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [51]:
#!pip install geocoder

In [67]:
#!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['Postal Code'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
    

Postal Code M3A has been retrieved. 102 codes left
Postal Code M4A has been retrieved. 101 codes left
Postal Code M5A has been retrieved. 100 codes left
Postal Code M6A has been retrieved. 99 codes left
Postal Code M7A has been retrieved. 98 codes left
Postal Code M9A has been retrieved. 97 codes left
Postal Code M1B has been retrieved. 96 codes left
Postal Code M3B has been retrieved. 95 codes left
Postal Code M4B has been retrieved. 94 codes left
Postal Code M5B has been retrieved. 93 codes left
Postal Code M6B has been retrieved. 92 codes left
Postal Code M9B has been retrieved. 91 codes left
Postal Code M1C has been retrieved. 90 codes left
Postal Code M3C has been retrieved. 89 codes left
Postal Code M4C has been retrieved. 88 codes left
Postal Code M5C has been retrieved. 87 codes left
Postal Code M6C has been retrieved. 86 codes left
Postal Code M9C has been retrieved. 85 codes left
Postal Code M1E has been retrieved. 84 codes left
Postal Code M4E has been retrieved. 83 codes le

In [68]:
#test:

df['Latitude'] = latitudes
df['Longitude'] = longitudes

df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7545,-79.3300
3,M4A,North York,Victoria Village,43.7276,-79.3148
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
9,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
11,M3B,North York,Don Mills,43.7450,-79.3590
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [69]:
df.shape

(103, 5)

In [70]:
#assess to check for missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Postal Code    103 non-null    object 
 1   Borough        103 non-null    object 
 2   Neighbourhood  103 non-null    object 
 3   Latitude       102 non-null    float64
 4   Longitude      102 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [73]:
df[df.Latitude.isnull()]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
114,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


**There is missing data in Latitude & Longitude columns.**
- This row will have to be dropped in order to generate our map

In [74]:
#drop row with missing lat and long info:
df.dropna(inplace=True)

In [75]:
#test:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 2 to 178
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Postal Code    102 non-null    object 
 1   Borough        102 non-null    object 
 2   Neighbourhood  102 non-null    object 
 3   Latitude       102 non-null    float64
 4   Longitude      102 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [76]:
# shape after drop:
df.shape

(102, 5)

In [ ]:
#use csv file instead and append to existing df:

#geo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')

In [ ]:
#df2 = df.join(geo_df, on='Postcode')

------------------------------

### Part 3: Explore and cluster the neighborhoods in Toronto.

**We need to get the latitude and longitude values of Toronto.**

In [77]:
#geopy library to get the latitude and longitude values:
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Create a map of Toronto with neighborhoods superimposed on top.**

In [78]:
# create map of Toronto using latitude and longitude values
map_ca = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ca)
    
map_ca

**let's simplify the above map and segment and cluster only the neighborhoods that are present in boroughs that contain the word Toronto**

In [82]:
#create a subset that only contains the boroughs that include the word 'Toronto':
df2 = df[df.Borough.str.contains("Toronto")].reset_index(drop=True)

In [84]:
df2.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
7,M6G,Downtown Toronto,Christie,43.6683,-79.4205
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378


In [85]:
df2.shape

(39, 5)

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them. 

1) Define Foursquare Credentials and Version:

In [87]:
CLIENT_ID = 'TSQEK2IRFXN5FJAR0WM3DKMCC0HF0MSCTZJNXTW2Z1GGNWHK' # your Foursquare ID
CLIENT_SECRET = '3RMPFCZ2KM21ULJ2NZXOJ4E4LPU3LV4GHWB5F2ZUCMXRI2JE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TSQEK2IRFXN5FJAR0WM3DKMCC0HF0MSCTZJNXTW2Z1GGNWHK
CLIENT_SECRET:3RMPFCZ2KM21ULJ2NZXOJ4E4LPU3LV4GHWB5F2ZUCMXRI2JE


2) Let's explore the first neighborhood in our dataframe.

In [90]:
#get the 1st neighbourhoods name:
df2.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

- There are two neighbourhoods here so let's rather use a row with one neighbourhood recorded instead of multiple:


In [91]:
#get the 4th neighbourhoods name:
df2.loc[3, 'Neighbourhood']

'St. James Town'

3) Get the latitude and longitude values for the neighbourhood 'St. James Town'.


In [93]:
neighborhood_latitude = df2.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[3, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6513, -79.3756.


4) Now, let's get the top 100 venues that are in St. James Town within a radius of 500 meters.

- First, let's create the GET request URL.

In [94]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TSQEK2IRFXN5FJAR0WM3DKMCC0HF0MSCTZJNXTW2Z1GGNWHK&client_secret=3RMPFCZ2KM21ULJ2NZXOJ4E4LPU3LV4GHWB5F2ZUCMXRI2JE&v=20180605&ll=43.6513,-79.3756&radius=500&limit=100'

In [95]:
#Send the GET request
results = requests.get(url).json()

- Get teh catagory of venues by defining a function:

In [96]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

- Clean the json and structure it into a pandas dataframe.

In [98]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
1,Hogtown Smoke,Food Truck,43.649287,-79.374689
2,Crepe TO,Creperie,43.650063,-79.374587
3,Versus Coffee,Coffee Shop,43.651213,-79.375236
4,Terroni,Italian Restaurant,43.650927,-79.375602


In [99]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

91 venues were returned by Foursquare.


- Create a function to repeat the same process to all the neighborhoods for boroughs that include teh word 'Toronto'

In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

- run the above function on each neighborhood and create a new dataframe with venues

In [101]:
#new df:
t_venues = getNearbyVenues(names=df2['Neighbourhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [102]:
#size of new df:
t_venues.shape

(1529, 7)

In [103]:
t_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


In [105]:
#how many venues were returned for each neighborhood?
t_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,92,92,92,92,92,92
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,58,58,58,58,58,58
Christie,12,12,12,12,12,12
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,23,23,23,23,23,23


In [106]:
#how many unique categories can be curated from all the returned venues?
print('There are {} uniques categories.'.format(len(t_venues['Venue Category'].unique())))

There are 215 uniques categories.


- Analyze Each Neighborhood

In [112]:
# one hot encoding
t_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighbourhood'] = t_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

t_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
#new df size:
t_onehot.shape

(1529, 215)

In [111]:
#columns:
t_onehot.columns

Index(['Yoga Studio', 'Accessories Store', 'American Restaurant',
       'Art Gallery', 'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant',
       'Athletics & Sports', 'BBQ Joint', 'Baby Store',
       ...
       'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Whisky Bar', 'Wine Bar', 'Wine Shop',
       'Wings Joint'],
      dtype='object', length=215)

- Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [113]:
t_grouped = t_onehot.groupby('Neighbourhood').mean().reset_index()
t_grouped.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.01087,0.021739,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.01087,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.010870
1,"Brockton, Parkdale Village, Exhibition Place",0.025641,0.00000,0.025641,0.000000,0.025641,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.025641
2,"Business reply mail Processing Centre, South C...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.066667
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00000,0.000000,0.000000,0.000000,0.017544,0.0,0.0,0.0,...,0.0,0.017544,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.017544,0.017544
4,Central Bay Street,0.000000,0.00000,0.000000,0.017241,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.017241,0.017241,0.0,0.017241,0.0,0.000000,0.000000


In [114]:
#new df size:
t_grouped.shape

(39, 216)

-  Print each neighborhood along with the top 5 most common venues

In [115]:
num_top_venues = 5

for hood in t_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = t_grouped[t_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0     Restaurant  0.13
1    Coffee Shop  0.13
2    Yoga Studio  0.07
3           Bank  0.07
4  Deli / Bodega  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1                 Bar  0.05
2                Café  0.05
3  Italian Restaurant  0.05
4   French Restaurant  0.04


----Central Bay Street----
                ve

#### Put the above data into a pd df:

In [116]:
#write a function to sort the venues in descending order.:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [117]:
#create the new dataframe and display the top 10 venues for each neighborhood.:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
t_venues_sorted = pd.DataFrame(columns=columns)
t_venues_sorted['Neighbourhood'] = t_grouped['Neighbourhood']

for ind in np.arange(t_grouped.shape[0]):
    t_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

t_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Bakery,Café,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Cocktail Bar,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Gift Shop,Mexican Restaurant,Liquor Store,Japanese Restaurant,French Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Yoga Studio,Bookstore,Martial Arts School,Japanese Restaurant,Italian Restaurant,Bank,Breakfast Spot,Department Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Bar,Speakeasy,Restaurant,Bank,Bakery,Park,French Restaurant
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Restaurant,Clothing Store,Café,Pizza Place,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Shoe Store


### Cluster the Neighbourhoods:

**Run k-means to cluster the neighborhood into 5 clusters**

In [119]:
#set number of clusters
kclusters = 5

t_grouped_clustering = t_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [120]:
# add clustering labels
t_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

t_merged = df2

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(t_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

t_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,2,Coffee Shop,Breakfast Spot,Electronics Store,Beer Store,Restaurant,Pub,Italian Restaurant,Event Space,Thai Restaurant,Theater
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,2,Sushi Restaurant,Coffee Shop,College Cafeteria,Hobby Shop,Bubble Tea Shop,Café,Ramen Restaurant,Portuguese Restaurant,College Theater,Creperie
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,2,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Café,Hotel,Cosmetics Shop,Movie Theater,Italian Restaurant,Diner
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,Gastropub,Cocktail Bar,Bakery,American Restaurant,Hotel,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,2,Pub,Health Food Store,Asian Restaurant,Gastropub,Bakery,Trail,Neighborhood,Diner,Eastern European Restaurant,Flower Shop


#### Visualize the resulting clusters.

In [121]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_merged['Latitude'], t_merged['Longitude'], t_merged['Neighbourhood'], t_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [122]:
t_merged.loc[t_merged['Cluster Labels'] == 0, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Playground,Candy Store,Coffee Shop,Athletics & Sports,Baby Store,Farmers Market,Event Space
10,Downtown Toronto,0,Music Venue,Harbor / Marina,Café,Park,Yoga Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
23,Central Toronto,0,Playground,Gym Pool,Park,Garden,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
29,Central Toronto,0,Park,Gym,Grocery Store,Thai Restaurant,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
33,Downtown Toronto,0,Playground,Park,Grocery Store,Candy Store,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 2

In [123]:
t_merged.loc[t_merged['Cluster Labels'] == 1, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Home Service,Yoga Studio,Dumpling Restaurant,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 3

In [124]:
t_merged.loc[t_merged['Cluster Labels'] == 2, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Electronics Store,Beer Store,Restaurant,Pub,Italian Restaurant,Event Space,Thai Restaurant,Theater
1,Downtown Toronto,2,Sushi Restaurant,Coffee Shop,College Cafeteria,Hobby Shop,Bubble Tea Shop,Café,Ramen Restaurant,Portuguese Restaurant,College Theater,Creperie
2,Downtown Toronto,2,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Café,Hotel,Cosmetics Shop,Movie Theater,Italian Restaurant,Diner
3,Downtown Toronto,2,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,Gastropub,Cocktail Bar,Bakery,American Restaurant,Hotel,Moroccan Restaurant
4,East Toronto,2,Pub,Health Food Store,Asian Restaurant,Gastropub,Bakery,Trail,Neighborhood,Diner,Eastern European Restaurant,Flower Shop
5,Downtown Toronto,2,Coffee Shop,Hotel,Bakery,Café,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Cocktail Bar,Deli / Bodega
6,Downtown Toronto,2,Coffee Shop,Middle Eastern Restaurant,Restaurant,Clothing Store,Café,Pizza Place,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Shoe Store
8,Downtown Toronto,2,Coffee Shop,Café,Gym,Restaurant,Hotel,Steakhouse,Thai Restaurant,American Restaurant,Sushi Restaurant,Asian Restaurant
9,West Toronto,2,Bakery,Park,Pool,Bus Line,Pizza Place,Bar,Bank,Brazilian Restaurant,Smoke Shop,Pharmacy
11,West Toronto,2,Bar,Coffee Shop,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Skating Rink


#### Cluster 4

In [125]:
t_merged.loc[t_merged['Cluster Labels'] == 3, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Photography Studio,Park,Lawyer,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
21,Central Toronto,3,Business Service,Park,Lawyer,Trail,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


#### Cluster 5

In [126]:
t_merged.loc[t_merged['Cluster Labels'] == 4, t_merged.columns[[1] + list(range(5, t_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,4,Park,Residential Building (Apartment / Condo),Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
